# RDD: Resilient Distributed Dataset

### Что такое RDD

* набор объектов, разбитых на разделы (`partitions`)
* часть Low Level API
* dataframe "компилируются" в RDD
* менее функционально полная абстракция spark

### Зачем нам нужны 

* для понимания концепции (как SQL)
* работы с legacy кодом
* если нужна функциональность, которой нет в Structured API (например, управление размещением данных по разделам)
* фокус все равно на Structured API

### Формальное определение RDD

Процитирую из книги - RDD внутренне характеризуется следующим набором атрибутов:

* списком разделов (`partitions`)
* фукнцией для вычисления каждого `split`
* списком зависимостей от других RDD
* (опционально) функцией `Partitioner` для `key-value RDD`
* (опционально) списком предпочтений узлов, на которых обрабатывать каждый `split` (например, список расположений блоков файла HDFS)

### Трансформации и действия

* та же модель, что и с dataframe
* набор трансформаций меньше (и он - другой)
* используется lazy evaluation 

### Создание RDD

* `spark.sparkConect` - точка входа в Low Level API
* `parallelize()` - создание RDD
* `textFile()` - создание RDD из файлов (каждая строка - элемент RDD)
* `wholeTextFiles()` - каждый файл - элемент RDD (например, наши сложные XML или JSON)

### Основные трансформации

* `distinct()` - оставить только уникальные элементы RDD
* `filter()` - отфильтровать элементы с помощью заданной функции
* `map()` - преобразовать элементы с помощью заданной функции
* `flatMap()` - преобразовать с добавлением элементов RDD
* `sort()` - упорядочить элементы RDD

### Основные действия

* `reduce()` - сворачивает элементы RDD, используя коммутативную и ассоциативную функцию (с двумя параметрами)
* `count()` - посчитать количество элементов в RDD
* `first()` - собрать первые N элементов RDD на драйвер
* `min()/max()` - найти максимум и минимум 
* `collect()/take()` - собрать/собрать N элементов RDD на драйвер
* `saveAsTextFile()` - сохранить RDD в виде текстового файла

### python и RDD

При использовании python-а и RDD происходит потеря производительности (в отличие от RDD + java или использования Structured API): RDD - это как использование python UDF для каждого элемента RDD

* сначала данные сериализуются в python структуры данных
* обрабатываем их на python-е
* сериализуем данные в формат JVM

### Практикуемся

In [51]:
import os
from pyspark.sql import SparkSession

os.environ["SPARK_HOME"] = "/home/mk/mk_win/projects/SparkEdu/lib/python3.5/site-packages/pyspark"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "pyspark-shell"

In [52]:
spark = SparkSession.builder.master("local").appName("spark_app").getOrCreate()

**Создание RDD**

Давайте создадим простейший RDD - слова файла (как строки)

In [53]:
wrds = spark.sparkContext.parallelize("this is spark and it is great".split(),3)

**Из файла**

Прочитаем файл и создадим RDD со строками файла

In [54]:
lines = spark.sparkContext.textFile("data/countries of the world.csv")
lines.first()

'Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service'

**Трансформации**

Начнем с фильтрации, давайте сделаем что-то со словами, ниже - разберем, что получилось и почему RDD не эффективны

In [55]:
def startsWithS(individual):
    return individual.startswith("s")

wrds.filter(lambda word: startsWithS(word)).collect()

['spark']

Обсуждение сделанного.

Что произошло:

* мы написали функцию на python (startsWithS)
* она была spark-ом как-то переведена на java (executor-ы "понимают" только java)
* на каждом узле кластера (где "живут" разделы нашего RDD) для каждого элемента раздела RDD была вызвана эта функция
* на вход был подан "переведенный" в python строку (десериализация) элемент раздела RDD с этого узла
* функция отработала, вернула True или False
* в раздел нового RDD попала часть элементов старого (возможно, произошла еще одна сериализация)
* все разделы собираются на драйвер (`collect`) и происходит еще одна десериализация (в типы python)

Обратите внимание - результат `collect()` - список строк (чисто python объект), т.е. никакого преобразования в объекты spark не происходит. В функции `startsWithS()` мы работаем со строками. Spark делает за нас все преобразования, но это снижает эффективность, нужно иметь это в виду

**dictinct()**

У нас есть повторяющиеся слова - удалим их

In [56]:
wrds.distinct().collect()

['it', 'and', 'is', 'spark', 'this', 'great']

**map()**

Преобразуем наши слова в пары - слово, длина

In [57]:
def getLen(individual):
    return (individual,len(individual))

wrds.map(getLen).collect()

[('this', 4),
 ('is', 2),
 ('spark', 5),
 ('and', 3),
 ('it', 2),
 ('is', 2),
 ('great', 5)]

**flatMap()**

Разобъем слова по символам (т.е. увеличим количество разделов - каждый символ = раздел) и соберем на `driver` первые 6 разделов

In [58]:
def toChars(individual):
    return list(individual)

wrds.flatMap(toChars).take(6)

['t', 'h', 'i', 's', 'i', 's']

**sort()**

Давайте упорядочим слова по их длине (по убыванию)

In [59]:
wrds.sortBy(lambda word: len(word) * -1).collect()

['spark', 'great', 'this', 'and', 'is', 'it', 'is']

**Действия**

мы уже видели `collect()/take()`, давайте посмотрим `reduce()`: оставим самое длинное слово из нашего набора

In [60]:
def wordLengthReducer(leftWord, rightWord):
    if len(leftWord) > len(rightWord):
        return leftWord
    else:
        return rightWord

wrds.reduce(wordLengthReducer)

'great'

**count()**

In [61]:
wrds.count()

7

**saveAsTextFile()**

Сохраним наши слова в виде "файла" - как обычно для Spark: будет создана директория, в которую будет записан RDD, каждый раздел - отдельный файл. Количество разделов мы тоже выведем, хотя это - 3 (мы именно так создавали RDD - см. выше).

In [62]:
!rm -rf data/words.txt
wrds.saveAsTextFile("data/words.txt")
wrds.getNumPartitions()

3

In [63]:
numbs = spark.sparkContext.parallelize([1.0, 5.0, 43.0, 10.0])
print(numbs.max()," as numbers")
print(numbs.max(key=str)," as strings")

43.0  as numbers
5.0  as strings


### Еще немного практики

трансформация `glom()`

Собирает все элементы раздела в список.

In [64]:
wrds.glom().collect()

[['this', 'is'], ['spark', 'and'], ['it', 'is', 'great']]

**Кэширование**

Как и с датафреймами - не сможем это эффективно использовать, но упомянуть стоит.

`cache()` - кэширует RDD (после выполнения любого следующего действия), после этого RDD уже не будет каждый раз исполняться, значения будут браться из кэша. 

После выполнения этого узла можно будет увидеть статус RDD здесь - localhost:4040/storage/


In [65]:
wrds.cache()
wrds.count()

7

Обратная операция - `unpersist()`

После ее выполнения кэш пропадет (см. localhost:4040/storage/)

In [66]:
wrds.unpersist()

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:195

**Использование RDD для небольших XML файлов**

Как один из вариантов практического использования RDD - работа с мелкими XML файлами (помните, мы разбирали проблематику в модуле про работу с источниками?).

Можно считать XML (в преобразованном, конечно, виде) в RDD, а дальше с помощью `map()` "вытащить" нужные поля в "плоскую" часть, оставив весь исходный XML в виде одного из полей. Потом эту "регулярную" структуру сохранить в реляционной таблице, например.

Это будет не очень эффективно - см выше - но для обработки относительно небольшого потока XML может и хватить. 

Как мысль.

In [67]:
dList = [ 
    { "id": 1, "details": { "name": "pete", "phone": 123 } },
    { "id": 2, "details": { "name": "mike", "phone": 999 } },
]
dictRdd = spark.sparkContext.parallelize(dList)

def mkRecord(el):
    return ( el["id"], el["details"]["name"], el)

dictRdd.map(mkRecord).collect()

[(1, 'pete', {'details': {'name': 'pete', 'phone': 123}, 'id': 1}),
 (2, 'mike', {'details': {'name': 'mike', 'phone': 999}, 'id': 2})]

**Пример вычисления числа Пи**

Давайте, наконец, разберем - что же происходило в том примере, который мы использовали для вычисления числа Пи (в начале - когда тестировали работоспособность Spark), код приведен ниже.

Теперь мы все знаем и можем объяснить - собственно, один из "смыслов" изучения RDD:

* создается простейщий RDD (один раздел, список чисел нужного диапазона)
* функция `filter()` позволяет нам выполнить код на питоне (вообще говоря - любой), в этом примере параметр (элемент RDD, для которого происходит вызов) не используется
* функция `inside()` написана на python, использует его инструментарий
* возвращает true или false, в зависимости от того, попали ли случайные числа в круг диаметром 2 или нет
* фильтруя мы оставляем в RDD только "попавшие" в круг его элементы
* `count()` посчитает число попавших в круг элементов
* далее - простая математика (которая геометрия)

Все просто! ("как хорошо уметь читать...")

In [68]:
import random
num_samples = 10000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = spark.sparkContext.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

3.1388


In [69]:
spark.stop()